In [8]:
import os
import configparser
from dataclasses import dataclass

import datasets

from nltk import sent_tokenize
from datasets import load_from_disk
from tokenizers import Tokenizer
from transformers import BertModel, AutoTokenizer, AutoModel, BertTokenizerFast

# Loading

In [13]:
file_name = "tokenizers/tokenizer_2_1000.json"

base_tokenizer = Tokenizer.from_file(file_name)
tokenizer = BertTokenizerFast(tokenizer_object=base_tokenizer)

In [4]:
root_dir = r"C:\Users\onurg\.cache\huggingface\datasets"
data_dir = "updated_wiki40b"

path = os.path.join(root_dir, data_dir, "long_small_dataset")

In [5]:
dataset = load_from_disk(path)
dataset

In [7]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

# Prototyping

In [11]:
small_dataset = dataset.select(range(100))

In [14]:
@dataclass
class Config:
    def __init__(self, max_sentence_len, max_doc_len):
      self.max_sentence_len = max_sentence_len
      self.max_doc_len = max_doc_len

args = Config(128,32)

In [15]:
def tokenize(example, tokenizer, args):
    # https://github.com/castorini/hedwig/blob/master/datasets/bert_processors/abstract_processor.py
    # https://github.com/abhishekkrthakur/bert-entity-extraction/blob/master/src/dataset.py

    def tokenize_helper(article, tokenizer, args):
        # TODO: comment
        sentences = [tokenizer.encode(sentence, add_special_tokens=False) for sentence in sent_tokenize(article)] 
        sentences = [sentence[:args.max_sentence_len - 2] for sentence in sentences]
        sentences = [[tokenizer.convert_tokens_to_ids("[CLS]")] + sentence + [tokenizer.convert_tokens_to_ids("[SEP]")] for sentence in sentences]

        sentence_lengths = [len(sentence) for sentence in sentences]
        mask = [[1]*sen_len for sen_len in sentence_lengths]

        return sentences, mask
 
    for i in range(1, 3):
        
        # example[f"article_{i}"] = [tokenizer.encode(sentence, 
        #                                             truncation=True,
        #                                             add_special_tokens=True,                                            
        #                                             max_length=args.max_sentence_len) for sentence in sent_tokenize(example[f"article_{i}"])]
                                                
        #print(example)

        example[f"article_{i}"], example[f"mask_{i}"] = tokenize_helper(example[f"article_{i}"], tokenizer, args)
        
    return example

In [16]:
new_dataset = small_dataset.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "args": args})

100%|██████████| 100/100 [00:02<00:00, 43.79ex/s]


In [1]:
new_dataset

NameError: name 'new_dataset' is not defined